# Chapter 17 - Reinforcement Learning

Reinforcement Learning (RL) is a suite of techniques to build machine learning systems that take decisions sequentially.

The key distinction between reinforcement learning and standard deep learning is that in standard deep learning the prediction of a trained model on one test datum does not affect the predictions on a future test datum; in reinforcement learning decisions at future instants (in RL, decisions are also called actions) are affected by what decisions were made in the past.

## 17.1. Markov Decision Process (MDP)

### 17.1.1. Definition of an MDP

A Markov decision process (MDP) is a model for how the state of a system evolves as different actions are applied to the system. An MDP is defined by the following components:
* The set of states $\mathcal{S}$ in the MDP.
* The set of actions $\mathcal{A}$ that an agent can take. Actions can change the current state of the agent to some other state within the set $\mathcal{S}$.
* We may not know how the agent's state changes exactly but only know it up to approximation. In this case, there is a transition probability function $T: \mathcal{S} \times \mathcal{A} \times \mathcal{S} \rightarrow [0, 1]$ such that $T(s, a, s') = P(s' \mid s, a)$ using the conditional probability of eaching a state $s'$ given that the robot was at state $s$ and took an action $a$. The transition function is a probability distribution and we therefore have $\sum_{s' \in \mathcal{S}} T(s, a, s') = 1$ for all $s \in \mathcal{S}$ and $a \in \mathcal{A}$.
* A reward function $R: \mathcal{S} \times \mathcal{A} \rightarrow \mathbb{R}$ that gives a reward to the agent for taking an action $a$ at state $s$. If the reward $r(s,a)$ is large, this indicates that taking the action $a$ at state $s$ is more useful to achieving the goal of the agent. The reward is designed by the user with the goal in mind.

### 17.1.2. Return and Discount Factor

A Markov decision process (MDP) is defined as
\begin{split}
\textrm{MDP}: (\mathcal{S}, \mathcal{A}, T, r).
\end{split}

When an agent starts at a particular state $s_0 \in \mathcal{S}$ and continues taking actions, the agent will end up in a trajectory
\begin{split}
\tau = (s_0, a_0, r_0, s_1, a_1, r_1, s_2, a_2, r_2, \ldots).
\end{split}

At each time step $t$, the agent is at a state $s_t$ and takes an action $a_t$, which results in a reward $r_t=r(s_t, a_t)$. The *return* of a trajectory is the total reward obtained by the agent along such a trajectory:
\begin{split}
R(\tau) = r_0 + r_1 + r_2 + \cdots.
\end{split}

The objective in reinforcement learning is to find a trajectory that has the largest *return*.

The sequence of states and actions in a trajectory can be infinitely long and the return of any such infinitely long trajectory will be infinite. In order to keep the reinforcement learning formulation meaningful even for such trajectories, we introduce the notion of a discount facotr $\gamma \in [0, 1)$ and define the *discounted return* as
\begin{split}
R(\tau) = r_0 + \gamma r_1 + \gamma^2 r_2 + \cdots = \sum_{t=0}^\infty \gamma^t r_t.
\end{split}

If $\gamma$ is very small, the rewards earned by the agent in the far future, say $t=1000$, are heavily discounted by the factor $\gamma^{1000}$. This encourages the agent to select short trajectories that achieve its goal.

For large values of the discount factor, say $\gamma=0.99$, the agent is encouraged to *explore* and then find the best trajectory to go to the goal state.

### 17.1.3. Discussion of the Markov Assumption

Markov systems are all systems where the next state $s_{t+1}$ is only a function of the current state $s_t$ and the action $a_t$ taken at the current state. In Markov systems, the next state does not depend on which actions were taken in the past or the states that the robot was at in the past.

## 17.2. Value Iteration

### 17.2.1. Stochastic Policy

A *stochastic policy* denoted as $\pi(a \mid s)$ (policy for short) is a conditional distribution over the actions $a \in \mathcal{A}$ given the state $s \in \mathcal{S}$, $\pi(a \mid s) \equiv P(a \mid s)$.

For example, if the agent has 4 actions $\mathcal{A} = \{ \textrm{up}, \textrm{down}, \textrm{left}, \textrm{right} \}$, then the policy at a state $s\in \mathcal{S}$ for such a set of actions $\mathcal{A}$ is a categorical distribution where the probabilities of the 4 actions could be $[0.3, 0.1, 0.4, 0.2]$; at some other state $s' \in \mathcal{S}$, the probabilities $\pi(a \mid s')$ could be $[0.6, 0.1, 0.1, 0.2]$.

Note that $\sum_a \pi(a \mid s) = 1$ for any state $s$.

A deterministic policy is a special case of a stochastic policy in that the distribution $\pi(a \mid s)$ only gives non-zero probability to one particular action, e.g., $[1,0,0,0]$ with 4 actions.

We will use $\pi(s)$ to denote the conditional distribution over the actions $a \in \mathcal{A}$ given the state $s \in \mathcal{S}$, $\pi(s) \equiv \pi(a \mid s)$.

### 17.2.2. Value Function

Suppose that the agent starts at a state $s_0$, and at each time step, it first samples an action from the policy $a_t \sim \pi(s_t)$ and takes this action to result in the next state $s_{t+1}$. 

The trajectory $\tau = (s_0, a_0, r_0, s_1, a_1, r_1, \ldots)$ can be different dpending upon which particular action $a_t$ is sampled at intermediate time steps.

The average *return* $R(\tau) = \sum_{t=0}^\infty \gamma^t r(s_t, a_t)$ of all such trajectories is defined as
\begin{split}
V^\pi(s_0) = E_{a_t \sim \pi(s_t)} \Big[ R(\tau) \Big] = E_{a_t \sim \pi(s_t)} \Big[ \sum_{t=0}^\infty \gamma^t r(s_t, a_t) \Big],
\end{split}
where $s_{t+1} \sim P(s_{t+1} \mid s_t, a_t)$ is the next state of the agent and $r(s_t, a_t)$ is the instantaneous reward obtained by taking action $a_t$ in state $s_t$ at time $t$. This is called the *value function* for the policy $\pi$.

In other words, the value of a state $s_0$ for a policy $\pi$, denoted by $V^\pi(s_0)$, is the expected $\gamma$-discounted *return* obtained by the agent if ti begins at state $s_0$ and take actions from the policy $\pi$ at each time step.

The trajectory $\tau$ can be broken down into the following components:
1. the first stage which corresponds to $s_0 \to s_1$ upon taking the action $a_0$, and
2. a second stage which is the trajectory $\tau' = (s_1, a_1, r_1, \ldots)$ thereafter.

The key idea behind all algorithms in reinforcement learning is that *the value of state $s_0$ can be written as the average reward obtained in the first stage and the value function averaged over all possible next stages $s_1$.

This arises from the Markov assumption: *the average return from the current state is the sum of the average return from the next state and the average reward of going to the next state*. Mathematically, we can rewrite the two stages as
\begin{split}
V^\pi(s_0) = r(s_0, a_0) + \gamma\ E_{a_0 \sim \pi(s_0)} \Big[ E_{s_1 \sim P(s_1 \mid s_0, a_0)} \Big[ V^\pi(s_1) \Big] \Big].
\end{split}

This is the foundation of the principle of *dynamic programming* upon which all reinforcement learning algorithms are based. The second stage gets two expectations, one over the choices of the action $a_0$ taken in the first stage using the sthchastic policy, and the other over the possible states $s_1$ obtained from the chosen action.

Using the transition probability in the Markov decision process, we can rewrite the above equation as
\begin{split}
V^\pi(s) = \sum_{a \in \mathcal{A}} \pi(a \mid s) \Big[ r(s,  a) + \gamma\  \sum_{s' \in \mathcal{S}} P(s' \mid s, a) V^\pi(s') \Big];\ \textrm{for all } s \in \mathcal{S}.
\end{split}

### 17.2.3. Action-Value Function

In practical implementations, we often maintain a quantity called the "action value" function which is closely related quantity to the value function. The action-value function $Q^\pi(s_0, a_0)$ is defined to be the average *return* of a trajectory that begins at state $s_0$ but when the action of the first stage is fixed to be $a_0$:
\begin{split}
Q^\pi(s_0, a_0) = r(s_0, a_0) + E_{a_t \sim \pi(s_t)} \Big[ \sum_{t=1}^\infty \gamma^t r(s_t, a_t) \Big],
\end{split}
where the summation inside the expectation is from $t=1,\ldots,\infty$ because the reward of the first stage is fixed in this case. We can also break down the trajectory into two parts and write
\begin{split}
Q^\pi(s, a) = r(s, a) + \gamma \sum_{s' \in \mathcal{S}} P(s' \mid s, a) \sum_{a' \in \mathcal{A}} \pi(a' \mid s')\ Q^\pi(s', a');\ \textrm{ for all } s \in \mathcal{S}, a \in \mathcal{A}.
\end{split}
This version for the action value function is the analog of
\begin{split}
V^\pi(s) = \sum_{a \in \mathcal{A}} \pi(a \mid s) \Big[ r(s,  a) + \gamma\  \sum_{s' \in \mathcal{S}} P(s' \mid s, a) V^\pi(s') \Big];\ \textrm{for all } s \in \mathcal{S}.
\end{split}

### 17.2.4. Optimal Stochastic Policy

Both the value function and the action-value function depend upon the policy that the agent chooses. The *optimal policy* is the policy that achieves the maximal average return:
\begin{split}
\pi^* = \underset{\pi}{\mathrm{argmax}} V^\pi(s_0).
\end{split}

Of all possible stochastic policies that the agent could have taken, the optimal policy $\pi^*$ achieves the largest average discounted *return* for trajectories starting from the initial state $s_0$. The value function and the action-value function for the optimal policy are denoted by $V^* \equiv V^{\pi^*}$ and $Q^* \equiv Q^{\pi^*}$, respectively.

For a deterministic policy where there is only one action that is possible under the policy at any given state, the optimal policy is also deterministic. In this case, the optimal policy is given as
\begin{split}
\pi^*(s) = \underset{a \in \mathcal{A}}{\mathrm{argmax}} \Big[ r(s, a) + \gamma \sum_{s' \in \mathcal{S}} P(s' \mid s, a)\ V^*(s') \Big].
\end{split}

The optimal action at state $s$ (for a deterministic policy) is the one that maximizes the sum of reward $r(s,a)$ from the first stage and the average return of the trajectories starting from the next state $s'$, averaged over all possible next states $s'$ from the second stage.

### 17.2.5. Principle of Dynamic Programming

From all the previous sections, we can summarize an algorithm to compute the optimal value function $V^*$ and the action-value function $Q^*$:
\begin{split}
V^*(s) = \sum_{a \in \mathcal{A}} \pi^*(a \mid s) \Big[ r(s,  a) + \gamma\  \sum_{s' \in \mathcal{S}} P(s' \mid s, a) V^*(s') \Big];\ \textrm{for all } s \in \mathcal{S}.
\end{split}

For a deterministic optimal policy $\pi^*$, since ther eis only one action that can be taken at state $s$, 
\begin{split}
V^*(s) = \mathrm{argmax}_{a \in \mathcal{A}} \Big\{ r(s,a) + \gamma \sum_{s' \in \mathcal{S}} P(s' \mid s, a) V^*(s') \Big\}
\end{split}
for all states $s \in \mathcal{S}$. This identity is called the “principle of dynamic programming”.

### 17.2.6. Value Iteration

The key idea behind value iteration is to use the principle of dynamic programming to compute the optimal value function $V^*$ and the optimal action-value function $Q^*$. We can initialize the value function to some arbitrary values $V_0(s)$ for all states $s \in \mathcal{S}$.

At the $k$-th iteration, the *value iteration algorithm* updates the value function as
\begin{split}
V_{k+1}(s) = \max_{a \in \mathcal{A}} \Big\{ r(s,  a) + \gamma\  \sum_{s' \in \mathcal{S}} P(s' \mid s, a) V_k(s') \Big\};\ \textrm{for all } s \in \mathcal{S}.
\end{split}

As $k \to \infty$, the value function estimated by the value iteration algorithm converges to the optimal value function iirespective of the initialization $V_0$, 
\begin{split}
V^*(s) = \lim_{k \to \infty} V_k(s);\ \textrm{for all states } s \in \mathcal{S}.
\end{split}

The same value iteration algorithm can be equivalently written using the action-value function as
\begin{split}
Q_{k+1}(s, a) = r(s, a) + \gamma \max_{a' \in \mathcal{A}} \sum_{s' \in \mathcal{S}} P(s' \mid s, a) Q_k (s', a');\ \textrm{ for all } s \in \mathcal{S}, a \in \mathcal{A}.
\end{split}

In this case, we initialize $Q_0(s, a)$ to some arbitrary values for all states $s \in \mathcal{S}$ and actions $a \in \mathcal{A}$. Thus, we have
\begin{split}
Q^*(s, a) = \lim_{k \to \infty} Q_k(s, a);\ \textrm{for all states } s \in \mathcal{S}, a \in \mathcal{A}.
\end{split}

### 17.2.7. Policy Evaluation

Value iteration is used to compute the optimal value function $V^{\pi^*}$ of the optimal deterministic policy $\pi^*$. We can apply similar iterative updates to compute the value function associated with any other, potentially stochastic, polynomial $\pi$.

We will initialize $V_0^{\pi}(s)$ to some arbitrary values for all states $s \in \mathcal{S}$. At the $k$-th iteration, we have the updates
\begin{split}
V^\pi_{k+1}(s) = \sum_{a \in \mathcal{A}} \pi(a \mid s) \Big[ r(s,  a) + \gamma\  \sum_{s' \in \mathcal{S}} P(s' \mid s, a) V^\pi_k(s') \Big];\ \textrm{for all } s \in \mathcal{S}.
\end{split}

This algorithm is known as *policy evaluation* and is useful to compute the value function given the policy. As $k \to \infty$, these updates comverge to the correct value function irrespective of the initialization $V_0$,
\begin{split}
V^\pi(s) = \lim_{k \to \infty} V^\pi_k(s);\ \textrm{for all states } s \in \mathcal{S}.
\end{split}

### 17.2.8. Implementation of Value Iteration

In [1]:
%matplotlib inline
import random
import numpy as np

from d2l import torch as d2l

In [2]:
# random number generator seed
seed = 1
# discount factor
gamma = 0.95
# number of iterations
num_iters = 10
# set the random seed for reproducibility
random.seed(seed)
np.random.seed(seed)

# set up the environment in OpenAI Gym
env_info = d2l.make_env('FrozenLake-v1', seed=seed)

In the FrozenLake environment, the agent moves on a $4\times 4$ grid (states) with actions that are "up", "down", "left", and "right". The environment contains a number of hole (H) cells and frozen (F) cells as well as a goal cell (G), all of which are unknown to the agent.

Assume that the agent has reliable actions, i.e., $P(s' \mid s, a) = 1$ for all $s, s' \in \mathcal{S}$ and $a \in \mathcal{A}$. If the agent reaches the goal, the trial ends and the agent receives a reward of 1 irrespective of the action; the reward at any other state is 0 for all actions.

The objective of the agent is to learn a policy that reaches the goal location (G) from a given start location (S) ($s_0$) to maximize the *return*.

The `value_iteration` function implements value iteration, where `env_info` contains MDP and environment related information and `gamma` is the discount factor.

In [3]:
def value_iteration(env_info, gamma, num_iters):
    """Value iteration algorithm."""
    env_desc = env_info['desc'] # 2d aray shows what each item means
    prob_idx = env_info['trans_prob_idx']
    nextstate_idx = env_info['nextstate_idx']
    reward_idx = env_info['reward_idx']
    num_states = env_info['num_states']
    num_actions = env_info['num_actions']
    mdp = env_info['mdp']

    # value function
    V = np.zeros((num_iters + 1, num_states))
    # state-action value function
    Q = np.zeros((num_iters + 1, num_states, num_actions))
    # policy
    pi = np.zeros((num_iters + 1, num_states))

    for k in range(1, num_iters + 1):
        for s in range(num_states):
            for a in range(num_actions):
                # calculate sum_s' P(s'|s,a) [r + gamma V_k(s')]
                for pxrds in mdp[(s, a)]:
                    # mdp(s,a): [(p1, next1, r1, d1), (p2, next2, r2, d2), ...]
                    pr = pxrds[prob_idx] # p(s' | s,a)
                    nextstate = pxrds[nextstate_idx] # next state
                    reward = pxrds[reward_idx] # reward

                    Q[k, s, a] += pr * (reward + gamma * V[k - 1, nextstate])

            # record max value and max action
            V[k, s] = np.max(Q[k, s, :])
            pi[k, s] = np.argmax(Q[k, s, :])

    d2l.show_value_function_progress(env_desc, V[:-1], pi[:-1])

In [ ]:
value_iteration(env_info=env_info, gamma=gamma, num_iters=num_iters)

## 17.3. Q-Learning

The *Q-Learning* algorithm is to learn the value function without necessarily knowing the MDP.

### 17.3.1. The Q-Learning Algorithm

The value iteration for the action-value function corresponds to the update
\begin{split}
Q_{k+1}(s, a) = r(s, a) + \gamma \sum_{s' \in \mathcal{S}} P(s' \mid s, a) \max_{a' \in \mathcal{A}} Q_k (s', a'); \ \textrm{for all } s \in \mathcal{S} \textrm{ and } a \in \mathcal{A}.
\end{split}

Implementing this update requires knowledge of the MDP, specifically the transition function $P(s' \mid s, a)$. The key idea behind Q-Learning is to replace the summation over all $s' \in \mathcal{S}$ in the above expression by a summation over the states visited by the agent.

### 17.3.2. An Optimization Problem underlying Q-Learning

Suppose that an agent uses a policy $pi_e(a \mid s)$ to take actions. It collects a dataset of $n$ trajectories of $T$ timesteps each $\{ (s_t^i, a_t^i)_{t=0,\ldots,T-1}\}_{i=1,\ldots, n}$.

The value iteration is a set of constraints that ties together the action-value $Q^*(s, a)$ of different states and actions to each other. We can implement an approximate version of value iteration using the data that the robot has collected using $pi_e$ as
\begin{split}
\hat{Q} = \min_Q \underbrace{\frac{1}{nT} \sum_{i=1}^n \sum_{t=0}^{T-1} (Q(s_t^i, a_t^i) - r(s_t^i, a_t^i) - \gamma \max_{a'} Q(s_{t+1}^i, a'))^2}_{\stackrel{\textrm{def}}{=} \ell(Q)}.
\end{split}

Recall that the value iteration algorithm is
\begin{split}
Q_{k+1}(s, a) = r(s, a) + \gamma \max_{a' \in \mathcal{A}} \sum_{s' \in \mathcal{S}} P(s' \mid s, a) Q_k (s', a');\ \textrm{ for all } s \in \mathcal{S}, a \in \mathcal{A}.
\end{split}

If the agent's policy $pi_e$ were equal to the optimal policy $pi^*$, and if it collected an infinite amount of data, then this optimization problem would be equivalent to the optimization problem underlying the value iteration algorithm. However, while value iteration requires us to know $P(s' \mid s, a)$, the optimization objective does not have this term.

This is because as the agent uses the policy $\pi_e$ to take an action $a_t^i$ at state $s_t^i$, the next state $s_{t+1}^i$ is a sample drawn from the transition function. So the optimization objective also has access to the transition function, but implicitly in terms of the data collected by the agent.

The variables of the optimization problem are the action-value function $Q(s, a)$ for all states $s \in \mathcal{S}$ and actions $a \in \mathcal{A}$. We can minimize the objective using gradient descent.

For every pair $(s_t^i, a_t^i)$ in the dataset, we have
\begin{split}
\begin{aligned}Q(s_t^i, a_t^i) &\leftarrow Q(s_t^i, a_t^i) - \alpha \nabla_{Q(s_t^i,a_t^i)} \ell(Q) \\&=(1 - \alpha) Q(s_t^i,a_t^i) - \alpha \Big( r(s_t^i, a_t^i) + \gamma \max_{a'} Q(s_{t+1}^i, a') \Big),
\end{aligned}
\end{split}
where $\alpha$ is the learning rate.

In practice, when the agent reaches the goal location, the trajectories end. The value of such a terminal state is zero because the agent does not take any further actions beyond this state. We can therefore modify our update to handle such states as
\begin{split}
Q(s_t^i, a_t^i) =(1 - \alpha) Q(s_t^i,a_t^i) - \alpha \Big( r(s_t^i, a_t^i) + \gamma (1 - \mathbb{1}_{s_{t+1}^i \textrm{ is terminal}} )\max_{a'} Q(s_{t+1}^i, a') \Big).
\end{split}
where $\mathbb{1}_{s_{t+1}^i \textrm{ is terminal}}$ is the indicator variable that is 1 if the state $s_{t+1}^i$ is a terminal state and 0 otherwise. The value of the state-action tuples $(s,a)$ that are not a part of the dataset is set to $-\infty$. This algorithm is called *Q-Learning*.

Given the solution of these updates $\hat{Q}$, which is an approximation of the optimal value function $Q^*$, we obtain the optimal deterministic policy corresponding to this value function using
\begin{split}
\hat{\pi}(s) = \mathrm{argmax}_{a} \hat{Q}(s, a).
\end{split}

### 17.3.3. Exploration in Q-Learning

Another key idea in Q-Learning is that the agent should explore the environment to collect data that is useful for learning the optimal value function. Typical implementations of Q-Learning tie together the current estimate of $Q$ and the the policy $\pi_e$ to set
\begin{split}
\pi_e(a \mid s) = \begin{cases}\mathrm{argmax}_{a'} \hat{Q}(s, a') & \textrm{with prob. } 1-\epsilon \\ \textrm{uniform}(\mathcal{A}) & \textrm{with prob. } \epsilon,\end{cases}
\end{split}
where $\epsilon$ is a hyperparameter that is typically set to a small value, called the "exploration parameter". The policy $\pi_e$ is called an exploration policy. This particular $\pi_e$ is called an $\epsilon$-greedy exploration policy because it chooses the optimal action (under the current estimate $\hat{Q}$) with probability $1-\epsilon$ but explores randomly with the remainder probability $\epsilon$.

We can also use the softmax exploration policy
\begin{split}
\pi_e(a \mid s) = \frac{e^{\hat{Q}(s, a)/T}}{\sum_{a'} e^{\hat{Q}(s, a')/T}},
\end{split}
where $T$ is a hyperparameter called the "temperature". A large value of $\epsilon$ in the $\epsilon$-greedy policy functions similarly to a large value of $T$ in the softmax policy.

Typical implementations of Q-Learning make one mini-batch update using a few state-action pairs in the collected dataset (typically the ones collected from the previous timestep of the agent) after taking every action using $\pi_e$. 

### 17.3.4. The "Self-Correcting" Property of Q-Learning

Consider a state $s$. If a particular action $a$ has a large value under the current estimate $\hat{Q}(s,a)$, then both the $\epsilon$-greedy and the softmax exploration policies have a larger probability of picking this action. If this action actually is *not* the ideal action, then the future states that arise from this action will have poor rewards. The next update of the Q-Learning objective will therefore reduce the value of $\hat{Q}(s,a)$, which will reduce the probability of picking this action the next time the agent visits state $s$.

* Bad actions, e.g., ones whose value is overestimated in $\hat{Q}(s,a)$, are explored by the agent but their value is correct in the next update of the Q-Learning objective.
* Good actions, e.g., ones whose value $\hat{Q}(s,a)$ is large, are explored more often by the agent and thereby reinforced.

This property can be used to show that Q-Learning can converge to the optimal policy even if it begins with a random policy $\pi_e$.

### 17.3.5. Implementation of Q-Learning

In [1]:
%matplotlib inline
import random
import numpy as np

from d2l import torch as d2l

In [ ]:
seed = 0  # Random number generator seed
gamma = 0.95  # Discount factor
num_iters = 256  # Number of iterations
alpha   = 0.9  # Learing rate
epsilon = 0.9  # Epsilon in epsilion gready algorithm
random.seed(seed)  # Set the random seed
np.random.seed(seed)

# Now set up the environment
env_info = d2l.make_env('FrozenLake-v1', seed=seed)

In [ ]:
def e_greedy(env, Q, s, epsilon):
    if random.random() < epsilon:
        return env.action_space.sample()

    else:
        return np.argmax(Q[s,:])

In [ ]:
def q_learning(env_info, gamma, num_iters, alpha, epsilon):
    env_desc = env_info['desc']  # 2D array specifying what each grid item means
    env = env_info['env']  # 2D array specifying what each grid item means
    num_states = env_info['num_states']
    num_actions = env_info['num_actions']

    Q  = np.zeros((num_states, num_actions))
    V  = np.zeros((num_iters + 1, num_states))
    pi = np.zeros((num_iters + 1, num_states))

    for k in range(1, num_iters + 1):
        # Reset environment
        state, done = env.reset(), False
        while not done:
            # Select an action for a given state and acts in env based on selected action
            action = e_greedy(env, Q, state, epsilon)
            next_state, reward, done, _ = env.step(action)

            # Q-update:
            y = reward + gamma * np.max(Q[next_state,:])
            Q[state, action] = Q[state, action] + alpha * (y - Q[state, action])

            # Move to the next state
            state = next_state
        # Record max value and max action for visualization purpose only
        for s in range(num_states):
            V[k,s]  = np.max(Q[s,:])
            pi[k,s] = np.argmax(Q[s,:])
    d2l.show_Q_function_progress(env_desc, V[:-1], pi[:-1])

q_learning(env_info=env_info, gamma=gamma, num_iters=num_iters, alpha=alpha, epsilon=epsilon)